<a href="https://colab.research.google.com/github/kangsunwoo827/rl_23_project_team5/blob/main/Final_project_team5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras

from collections import deque
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import gym
from gym import spaces

In [ ]:
##########윷놀이 환경 설정#############
# 출발 전 말은 0, 나간 말은 30번째로.
#
#   11 10  9  8  7  6
#   12 26       21  5
#   13    27  22    4
#           23
#   14   24   28    3
#   15 25        29 2
#   16 17 18 19 20  1
#

In [ ]:
empty_board='''
 -----------------------
| {}  {}  {}  {}  {}  {}|
|                       |
| {}   {}        {}   {}|
|        {}    {}       |
| {}                  {}|
|           {}          |
| {}                  {}|
|        {}    {}       |
| {}   {}        {}   {}|
|                       |
| {}  {}  {}  {}  {}  {}|
 -----------------------
 '''

board2num={
    11:0,10:1,9:2,8:3,7:4,6:5,
    12:6,26:7,       21:8,5:9,
          27:10,   22:11,
    13:12,               4:13,
              23:14,
    14:15,               3:16,
            24:17,28:18,
    15:19, 25:20,  29:21,2:22,
    16:23,17:24,18:25,19:26,20:27,1:28
 }

num2yut={
    1:'도',2:'개',3:'걸',4:'윷',5:'모',-1:'빽도'
}

In [ ]:
class YutEnv(gym.Env): # gym.Env를 상속
    def __init__(self,silent):
      self.silent=silent
      #뒤집어질 확률
      self.p=0.6
      self.p_lst=[
          3*(self.p**1)*((1-self.p)**3),
          6*(self.p**2)*((1-self.p)**2),
          4*(self.p**3)*((1-self.p)**1),
          1*(self.p**4)*((1-self.p)**0),
          1*(self.p**0)*((1-self.p)**4),
          1*(self.p**1)*((1-self.p)**3),
          ]
      #도 개 걸 윷 모 빽도

      
      self.pos= np.zeros(8,dtype=int)
      self._make_board()
      self.done=False
      self.team=np.random.choice([1,-1])
      self.bonus=0
      self.turn_num=0

      self.high=np.ones(249)+np.eye(249)[-1]*4
      self.action_space = spaces.Discrete(4)
      self.observation_space = spaces.Box(low=np.zeros(249),high=self.high,dtype=int)
      


    def step(self, action):
        self.turn_num+=1
        if not self.silent:
          print('{}번째 차례!'.format(self.turn_num))
          print('팀 {}의 차례입니다.'.format(self._num2team(self.team)))
          print('두구두구두구.... {}!!'.format(num2yut[self.move_num]))
        #random policy
        if not 0<=action<=3:
          raise Exception('action이 잘못됐습니다. cur_action is {}'.format(action))

        
        prev_pos=self.pos[action]
        next_pos=self._move(self.pos[action],self.move_num)

        if not self.silent:
          print('{}번째 말을 옮깁니다.'.format(action))
          print('이동 후 pos : ',self.pos)
        #윷, 모인 경우 한번 더!
        if self.move_num>=4:
          self.bonus=1
          
        for i in range(4):
          #업은 경우
          # i=i*self.team
          if prev_pos>0 and self.pos[i]==prev_pos:
            self.pos[i]=next_pos

          #잡은경우
          if 0<next_pos<30 and self.pos[-(i+1)]==next_pos:
            if not self.silent:
              print('잡았습니다!')
            self.pos[-(i+1)]=0
            self.bonus=1

        self.pos[action]=next_pos


        self._check_end()

        reward = self.bonus #정의해야함.

        #팀 바꾸기
        if self.bonus:
          self.bonus=0
          if not self.silent:
            print('한번 더!')
        else:
          self.pos=self.pos[::-1]
          self.team*=-1

        
        self._make_board()

        obs=self._get_state()
        

        return obs, reward, done, None

      
    def _check_end(self):
      if sum(self.pos[:4])==120 or sum(self.pos[-4:])==120:
        self.done=True

    def _move(self, prev_pos, move_num):
      #일단 빽도는 고려하지 않음.
      for i in range(move_num):
        next_pos=self._move_one(prev_pos,short_cut=bool(i==0))
        prev_pos=next_pos
      return next_pos

    def _move_one(self,prev_pos,short_cut=False):
      if short_cut:
        if prev_pos==6:
          return 21
        elif prev_pos==11:
          return 26
        elif prev_pos==23:
          return 28
      if prev_pos in [1,30]:
        return 30
      elif prev_pos==0:
        return 2
      elif prev_pos==29:
        return 1
      else:
        return prev_pos+1

    def reset(self):
      self.pos= np.zeros(8,dtype=int)
      self._make_board()
      self.done=False
      self.team=np.random.choice([1,-1])
      self.bonus=0
      self.turn_num=0
      return self._get_state()


    def _make_board(self):
      # 앞에 4개는 우리팀 말들의 위치, 뒤 4개는 상대팀 말들의 위치
      self.board = np.array([np.eye(31, dtype=int)[self.pos[i]] for i in range(8)])
      
      return self.board
      
    def _throw_yut(self):
      #yut_num은 던졌을 때 뒤집어진 개수. 0이면 모, 4면 윷
      self.move_num=np.random.choice([1,2,3,4,5,-1], p=self.p_lst)

      #빽도 없애는 코드
      if self.move_num==-1:
        self.move_num=1
      
      return self.move_num

    def _get_state(self):
      #현재 보드 판 상태와 내가 움직여야 하는 개수 return
      return np.append(self.board.flatten(),self._throw_yut())

    def _num2team(self,team):
      return "★" if team>0 else '■'
    
    def render(self):
      #내 말은 ● 상대 말은 X로 표시
      base=['◎']+['○']*4+['◎']+['○']*8+['◎']+['○']*8+['◎']+['○']*4+['◎']
      
      for i in range(4):
        if 0<self.pos[i]<30:
          base[board2num[self.pos[i]]]=self._num2team(self.team)
        if 0<self.pos[-(i+1)]<30:
          base[board2num[self.pos[-(i+1)]]]=self._num2team(-self.team)

      if not self.silent:
        print(empty_board.format(*base))
      return empty_board.format(*base)

    def show_result(self):
      if not self.silent:
        print("team{} is win!".format(self._num2team(self.team)))
      return self.team

      

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
def random_policy(state):
  possible_action=[]
  for i in range(4):

    if not state[31*(i+1)-1]==1: #도착한 말이 아니라면
      possible_action.append(i)
  return np.random.choice(possible_action)

env=YutEnv(silent=True)
env.render()
print('==')


==


In [ ]:
# res=[]
# for iter in range(1000):
#   done=False
#   state=env.reset()
#   while not env.done:
#     action=random_policy(state)
#     state,return_value, done = env.step(action)

#   winner=env.show_result()
#   res.append(winner)

# print(res.count(1))
# print(res.count(-1))

In [ ]:
# def play(show_number):
#     env = YutEnv(show_number=show_number)
#     agents = [HumanAgent('O'),
#               HumanAgent('X')]
#     episode = 0
#     while True:
#         state = env.reset()
#         _, mark = state
#         done = False
#         env.render()
#         while not done:
#             agent = agent_by_mark(agents, next_mark(mark))
#             env.show_turn(True, mark)
#             ava_actions = env.available_actions()
#             action = agent.act(ava_actions)
#             if action is None:
#                 sys.exit()

#             state, reward, done, info = env.step(action)

#             print('')
#             env.render()
#             if done:
#                 env.show_result(True, mark, reward)
#                 break
#             else:
#                 _, mark = state
#         episode += 1

In [ ]:
def policy_by_team(policies,team):
  return policies[0] if team==1 else policies[1]
policies=[random_policy,random_policy]

def play(iteration_num,policies=[random_policy,random_policy]):
  res=[]
  for iter in range(iteration_num):
    done=False
    state=env.reset()
    while not env.done:
      cur_policy=policy_by_team(policies,env.team)
      action=cur_policy(state)
      state,return_value, done = env.step(action)

    winner=env.show_result()
    res.append(winner)

  print('team ★ win :', res.count(1))
  print('team ■ win :', res.count(-1))


In [ ]:
# play(100)

**DQN START**

In [ ]:
# define neural net Q_\theta(s,a) as a class

class Qfunction(keras.Model):
    
    def __init__(self, obssize, actsize, hidden_dims):
        """
        obssize: dimension of state space
        actsize: dimension of action space
        hidden_dims: list containing output dimension of hidden layers 
        """
        super(Qfunction, self).__init__()

        # Layer weight initializer
        initializer = keras.initializers.RandomUniform(minval=-1., maxval=1.)

        # Input Layer
        self.input_layer = keras.layers.InputLayer(input_shape=(obssize,))
        
        # Hidden Layer
        self.hidden_layers = []
        for hidden_dim in hidden_dims:
            # TODO: define each hidden layers
            layer = keras.layers.Dense(hidden_dim, activation='relu',
                                      kernel_initializer=initializer)
            self.hidden_layers.append(layer) 
        
        # Output Layer : 
        # TODO: Define the output layer.
        self.output_layer = keras.layers.Dense(actsize) 
    
    @tf.function
    def call(self, states):
        # Connect the layers and pass the input tensor through them
        x = self.input_layer(states)
        for layer in self.hidden_layers:
            x = layer(x)
        output = self.output_layer(x)
        
        return output


In [ ]:
# Wrapper class for training Qfunction and updating weights (target network) 

class DQN(object):
    
    def __init__(self, obssize, actsize, hidden_dims, optimizer):
        """
        obssize: dimension of state space
        actsize: dimension of action space
        optimizer: 
        """
        self.qfunction = Qfunction(obssize, actsize, hidden_dims)
        self.optimizer = optimizer
        self.obssize = obssize
        self.actsize = actsize

    def _predict_q(self, states, actions):
        """
        states represent s_t
        actions represent a_t
        """
        q_values = self.qfunction(states)

        action_indices = tf.stack([tf.range(tf.shape(actions)[0]), actions], axis=-1)
        q_values = tf.gather_nd(q_values, action_indices)
        
        return q_values

    def _loss(self, Qpreds, targets):
        """
        Qpreds represent Q_\theta(s,a)
        targets represent the terms E[r+gamma Q] in Bellman equations

        This function is OBJECTIVE function
        """
        return tf.math.reduce_mean(tf.square(Qpreds - targets))

    
    def compute_Qvalues(self, states):
        """
        states: numpy array as input to the neural net, states should have
        size [numsamples, obssize], where numsamples is the number of samples
        output: Q values for these states. The output should have size 
        [numsamples, actsize] as numpy array
        """
        inputs = np.atleast_2d(states.astype('float32'))
        return self.qfunction(inputs)


    def train(self, states, actions, targets):
        """
        states: numpy array as input to compute loss (s)
        actions: numpy array as input to compute loss (a)
        targets: numpy array as input to compute loss (Q targets)
        """
        with tf.GradientTape() as tape:
            Qpreds = self._predict_q(states, actions)
            loss = self._loss(Qpreds, targets)
        variables = self.qfunction.trainable_variables
        gradients = tape.gradient(loss, variables)
        self.optimizer.apply_gradients(zip(gradients, variables))
        return loss

    def update_weights(self, from_network):
        """
        We need a subroutine to update target network 
        i.e. to copy from principal network to target network. 
        This function is for copying  𝜃←𝜃target 
        """
        
        from_var = from_network.qfunction.trainable_variables
        to_var = self.qfunction.trainable_variables
        
        for v1, v2 in zip(from_var, to_var):
            v2.assign(v1)

In [ ]:
# Implement replay buffer
class ReplayBuffer(object):
    
    def __init__(self, maxlength):
        """
        maxlength: max number of tuples to store in the buffer
        if there are more tuples than maxlength, pop out the oldest tuples
        """
        self.buffer = deque()
        self.number = 0
        self.maxlength = maxlength
    
    def append(self, experience):
        """
        this function implements appending new experience tuple
        experience: a tuple of the form (s,a,r,s^\prime)
        """
        self.buffer.append(experience)
        self.number += 1
        if(self.number > self.maxlength):
            self.pop()
        
    def pop(self):
        """
        pop out the oldest tuples if self.number > self.maxlength
        """
        while self.number > self.maxlength:
            self.buffer.popleft()
            self.number -= 1
    
    def sample(self, batchsize):
        """
        this function samples 'batchsize' experience tuples
        batchsize: size of the minibatch to be sampled
        return: a list of tuples of form (s,a,r,s^\prime)
        """
        inds = np.random.choice(len(self.buffer), batchsize, replace=False)
        return [self.buffer[idx] for idx in inds]

<>:14: DeprecationWarning: invalid escape sequence '\p'


In [ ]:
################################################################################
# TODO: Set the required parameters. All parameters can be tuned by yourself.
lr = 5e-4  # learning rate for gradient update 
batchsize = 64  # batchsize for buffer sampling
maxlength = 10000  # max number of tuples held by buffer
tau = 100  # time steps for target update
episodes = 3000  # number of episodes to run
initialize = 500  # initial time steps before start updating
epsilon = 1  # constant for exploration
gamma = 0.99  # discount
hidden_dims = [24,24] # hidden dimensions
epsilon_decay=0.95
min_epsilon=0.005
################################################################################

# initialize environment
env = YutEnv(silent=True)
obssize = env.observation_space.low.size
actsize = env.action_space.n

# optimizer
optimizer = keras.optimizers.Adam(learning_rate=lr)

# initialize networks
Qprincipal = DQN(obssize, actsize, hidden_dims, optimizer)
Qtarget = DQN(obssize, actsize, hidden_dims, optimizer)

# initialization of buffer
buffer = ReplayBuffer(maxlength)

################################################################################
# TODO: Complete the main iteration

In [ ]:

# TODO: Complete the main iteration
# CartPole-v0 defines "solving" as getting average reward of 195.0 over 100 consecutive trials.

rrecord = []
totalstep = 0
for ite in range(episodes):
    obs = env.reset()
    done = False
    rsum = 0
    time=0
    if epsilon > min_epsilon:
      epsilon *= epsilon_decay

    while not done:
        # Choose action
        #별만 학습
        if env.team==1:
          if np.random.uniform() < epsilon:
              # action = env.action_space.sample()
              action = random_policy(obs)
          else:
              q_values = Qprincipal.compute_Qvalues(obs)
              action = np.argmax(q_values)
        else:
          action = random_policy(obs)

        next, reward, done, _ = env.step(action)


        if env.team==1
          buffer.append((obs, action, reward, next))

          obs = next
          rsum += reward
          time += 1
          totalstep += 1
          
          if totalstep > initialize:
              batch = buffer.sample(batchsize)
              states = np.array([b[0] for b in batch])
              actions = np.array([b[1] for b in batch])
              rewards = np.array([b[2] for b in batch])
              next_states = np.array([b[3] for b in batch])

              next_qvalues = Qtarget.compute_Qvalues(next_states)
              targets = rewards + gamma * np.max(next_qvalues, axis=1)

              Qprincipal.train(states, actions, targets)
              if totalstep % tau == 0:
                  Qtarget.update_weights(Qprincipal)

################################################################################

    ## DO NOT CHANGE THIS PART! 
    rrecord.append(rsum)
    if ite % 10 == 0:
        print('iteration {} average reward {}'.format(ite, np.mean(rrecord[-10:])))
    
    ave100 = np.mean(rrecord[-100:])   
    if  ave100 > 195.0:
        print("Solved after %d episodes."%ite)
        break